# Horovod Distributed Training with SageMaker TensorFlow script mode.

Horovod is a distributed training framework based on MPI. You can find more details at [Horovod README](https://github.com/uber/horovod).

Horovod Distributed Training can be perfomed on SageMaker using the SageMaker Tensorflow container. SageMaker creates the MPI environment and executes the `mpirun` command to execute the training script.

MPI environment for Horovod can be configured by following flags in SageMaker SDK:

* ``enabled (bool)``: If set to ``True``, the MPI setup is performed and ``mpirun`` command is executed.
* ``processes_per_host (int) [Optional]``: Number of processes MPI should launch on each host. Note, this should not be greater than the available slots on the selected instance type.
* ``custom_mpi_options (str) [Optional]``: Additional command line arguments that we need to pass to ``mpirun``.

In this example notebook, we will create a MNIST horovod training job.

## Set up the environment

In [ ]:
import sagemaker
import os
from sagemaker.utils import sagemaker_timestamp
from sagemaker.tensorflow import TensorFlow

from sagemaker import get_execution_role

sagemaker_iam_role = get_execution_role()
sagemaker_iam_role = "SageMakerRole"
train_script = "mnist_hvd.py"
instance_count = 2

## Construct a script for horovod distributed training

In [ ]:
!cat 'mnist_hvd.py'

## Test locally using SageMaker Python SDK TensorFlow Estimator

You can use the SageMaker Python SDK TensorFlow estimator to easily train locally and in SageMaker.

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments. Just change your estimator's train_instance_type to local or local_gpu. For more information, see: https://github.com/aws/sagemaker-python-sdk#local-mode.

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU). Running following script will install docker-compose or nvidia-docker-compose and configure the notebook environment for you.

**Note**: You can only run a single local notebook at a time.

In [ ]:
!/bin/bash ./setup.sh

To train locally, you set train_instance_type to local:

In [ ]:
train_instance_type='local'

MPI can be configured by setting it to `true` in `distributions`

In [ ]:
distributions = {'mpi': {'enabled': True}}

Now, we create the Tensorflow estimator passing the `train_instance_type` and `distribution`

In [ ]:
estimator_local = TensorFlow(entry_point=train_script,
                       role=sagemaker_iam_role,
                       train_instance_count=instance_count,
                       train_instance_type=train_instance_type,
                       script_mode=True,
                       framework_version='1.12',
                       distributions=distributions,
                       base_job_name='hvd-mnist-local')

Call `fit()` to start the local training 

In [ ]:
estimator_local.fit()

## Train in SageMaker

After you test the training job locally, now run it on SageMaker:

First, change the instance type from `local` to the valid ec2 instance type. 

In [ ]:
train_instance_type='ml.c4.xlarge'

You can also provide your custom MPI options by passing in the `custom_mpi_options` field of `distribution` dictionary that will be added to the `mpirun` command executed by SageMaker:

In [ ]:
distributions = {'mpi': {'enabled': True, "custom_mpi_options": "-verbose --NCCL_DEBUG=INFO"}}

Now, we create the Tensorflow estimator passing the `train_instance_type` and `distribution` to launche training in sagemaker.

In [ ]:
estimator = TensorFlow(entry_point=train_script,
                       role=sagemaker_iam_role,
                       train_instance_count=instance_count,
                       train_instance_type=train_instance_type,
                       script_mode=True,
                       framework_version='1.12',
                       distributions=distributions,
                       base_job_name='hvd-mnist')

Call `fit()` to start the training

In [ ]:
estimator.fit()

##  Horovod training in SageMaker using multiple CPU/GPU

To enable mulitiple CPU/GPU horovod training, you have to set the `processes_per_host` field in `mpi` section of `distribution` dictionary to the desired value of processes that will be executed per instance.

In [ ]:
distributions = {'mpi': {'enabled': True, "processes_per_host": 2}}

Now, we create the Tensorflow estimator passing the `train_instance_type` and `distribution`

In [ ]:
estimator = TensorFlow(entry_point=train_script,
                       role=sagemaker_iam_role,
                       train_instance_count=instance_count,
                       train_instance_type=train_instance_type,
                       script_mode=True,
                       framework_version='1.12',
                       distributions=distributions,
                       base_job_name='hvd-mnist-multi-cpu')

Call `fit()` to start the training

In [ ]:
estimator.fit()

## Horovod Training in SageMaker using VPC environment

Providing a VPC improves the network throught of the training job and considerable increases the performance and stability of Horovod training jobs.

This can be done by supplying subnets and security groups to the job launching scripts. We will use the default VPC configuration for this example.

Detailed explanation on how to configure VPC for SageMaker training can be found [here](https://github.com/aws/sagemaker-python-sdk#secure-training-and-inference-with-vpc).

In [ ]:
import boto3

ec2 = boto3.client('ec2')

default_vpc = [vpc['VpcId'] for vpc in ec2.describe_vpcs()['Vpcs'] if vpc["IsDefault"] == True][0]

default_security_groups = [group["GroupId"] for group in ec2.describe_security_groups()['SecurityGroups'] \
                   if group["GroupName"] == "default" and group["VpcId"] == default_vpc]

default_subnets = [subnet["SubnetId"] for subnet in ec2.describe_subnets()["Subnets"] \
                  if subnet["VpcId"] == default_vpc and subnet['DefaultForAz']==True]

print("Using default VPC:", default_vpc)
print("Using default security group:", default_security_groups)
print("Using default subnets:", default_subnets)

Now, we create the Tensorflow estimator passing the train_instance_type and distribution

In [ ]:
estimator = TensorFlow(entry_point=train_script,
                       role=sagemaker_iam_role,
                       train_instance_count=instance_count,
                       train_instance_type=train_instance_type,
                       script_mode=True,
                       framework_version='1.12',
                       distributions=distributions,
                       security_group_ids=default_security_groups,
                       subnets=default_subnets,
                       base_job_name='hvd-mnist-vpc')

Call `fit()` to start the training

In [ ]:
estimator.fit()